<img src="uavsar.png" width="250" align="right" /> <br> 
# SAR Application: Fire Extent Detection and Mapping
## I. How to create SAR quicklook images of fire extent?
Polarimetric Synthetic Aperture Radar (PolSAR) products can be useful to map fire extent, particularly in the polarizations HH (horizontal transmit and horizontal receive), HV (horizontal transmit and vertical receive), and if possible VV (vertical transmit and vertical receive). To view the PolSAR products, Pauli decomposition can be used for quad-pol data, or HH/HV and VV/VH for dual-pol images. It is also useful to use the same scale for "before" and "after" image to facilitate comparison. 

Grayscale coherence correlation products can also highlight areas of change, and a coherence map can be used for immediate response after a fire to highlight areas of most severe damage.
## II. How to interpret SAR images of fire extent?
To interpret PolSAR images of burned areas, compare the fire scar with vegetation in the same area. Areas impacted by fire should show increase in HH component (bare ground) and lower HV component (vegetation). 

For interpreting coherence products, compare image pairs with increasing temporal separation up to a maximum that is determined empirically. After a certain time between acquisitions, the entire image is decorrelated. Coherence product values range 0-1, where areas of decorrelation or disturbance approach 0. Close to black areas are normally open water bodies that decorrelate the fastest, dark gray areas are regions decorrelated due to fire, and mid-gray and white have higher coherence or experienced less disturbance.

## UAVSAR Fire Example
Uninhabited Aerial Vehicle Synthetic Aperture Radar (UAVSAR) is an airborne radar instrument mounted on NASA's C-20A research aircraft. To gather information about the features on Earth, UAVSAR sends radio waves to the ground and measures the signals that reflect back to the sensor. When flown repeatedly over the same area, UAVSAR can measure small changes in the Earth's surface between flights by exploiting differences in the reflected signal.

For example, the radar regularly flies over California's San Andreas Fault to monitor ground displacement and earthquake activity. The coverage from these flights can also be used as baselines to map other types of changes on the surface, such as burn scars from a fire. After a fire, UAVSAR's radar signals bounce off the bare, freshly burned ground in a different way than denser pre-fire vegetation. Using these differences in the signal's interaction with the ground from pre and post fire, UAVSAR can be used to map burn scar extent. This information can be relevant for disaster response, since vegetation loss on hillsides may put these areas at risk of mudslides during rainstorms, and characterizing the spread of fires can help scientists understand a a fire's long-term effects.
#### La Tuna Fire
From September 1st through September 9th 2017, the La Tuna Fire burned over 7,000 acres of the Verdugo Mountains in Los Angeles, California, making it the city's largest wildfire in the last 50 years. The photos below were taken in the Verdugo Mountains after the fire and show the differences in vegetation density and height between unburned (left image) and burned (right image) regions.
<img src="latuna_not_burned.jpg" width="400" align="left"/><img src="latuna_burned.jpg" width="400" align="center"/>

### RGB Comparison
The distinction between burned and unburned regions is also clearly visible in the UAVSAR images. This interactive map displays two Pauli Decomposition RGB UAVSAR images of line SanAnd_08525, acquired June 24, 2014 (Pre-fire) and November 02, 2017 (Post-fire). 

The overlay of different polarization channels allows users to visually classify a scene by its backscattering mechanism, such as surface scattering (strong HH and VV return), volume scattering (strong HV return) and double-bounce scattering (strong HH return). Thus in RGB images, areas dominated by green (HV) intensity are typically vegetated areas. Areas dominated by shades of pink (HH+HV) intensity are typically inundated forests, vegetated fields, or urban areas. Black and dark grey areas are usually smooth surfaces (roads, open water, smooth bare ground) where there is little radar backscatter.   

The fire scar on the Western side of the mountains is evident, as volume scattering from vegetation becomes dominated by specular scattering from the burnt, bare ground in the UAVSAR image after the fire.

In [7]:
from ipyleaflet import (Map, basemaps, basemap_to_tiles, SplitMapControl, ImageOverlay, ScaleControl,
                        projections, LayersControl, FullScreenControl, LocalTileLayer, LegendControl)
import ipywidgets as widgets
from traitlets import Bool
from ipywidgets import AppLayout, Button, Layout, HTML
from pathlib import Path
from typing import *
import os

In [9]:
# Interactive Map Inputs
image1 = "2014_verdugo/{z}/{x}/{y}.png"
image2 = "2017_verdugo/{z}/{x}/{y}.png"
image3 = "verdugo_coherence/{z}/{x}/{y}.png"

class CustomLocalLayer(LocalTileLayer):
    tms = Bool(True).tag(sync=True, o=True)

m = Map(basemap=basemap_to_tiles(basemaps.Esri.WorldImagery),
    center=(34.22, -118.3),
    zoom=12
    
)

layer = CustomLocalLayer(tms=True, path=image1, name="June 2014 (Pre-Fire)")
layer2 = CustomLocalLayer(tms=True, path=image2, name="November 2017 (Post-Fire)")
layer3 = CustomLocalLayer(tms=True, path=image3, name="Coherence")

## Ipyleaflet Controls
control = SplitMapControl(left_layer=layer, right_layer=layer2)
legend = LegendControl({"HH-VV (Double Bounce)":"#FF0000", "HV+VH (Volume Scattering)":"#008000", "HH+VV (Specular Scattering)":"#0000FF"}, name="RGB Channels", position="topright")
description = LegendControl({"Bare ground":"#261947", "Urban (double bounce)":"#e8cef5", "Vegetation":"#7eab91"}, name="Interpretation", position="topright")
m.add_control(legend)
m.add_control(description)
m.add_control(control)
m.add_control(LayersControl())
m.add_control(FullScreenControl())
m.add_control(ScaleControl(position='bottomleft', metric=True, imperial=False, maxwidth=500))

m

Map(center=[34.22, -118.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

### Coherence
Radar coherence images measure how well the phase signal is preserved between UAVSAR acquisitions and can provide another method to detect areas of change. In a coherence map, darker areas have lower coherence or have decorrelated between acquisitions. For a fire application, areas of lower coherence should correspond with the fire scar extent since changes in vegetation and other associated damage from the fire will cause decorrelation. 

UAVSAR images from before and after the La Tuna Fire have a long temporal baseline (or separation in time between UAVSAR acquisitions). Therefore, other factors associated with the passing of time contributed to decorrelation, and the coherence map became too noisy to use for determining fire extent. Instead, we will look at the 2018 Thomas Fire, which has a much shorter temporal baseline. 

#### Thomas Fire
UAVSAR acquired data November 2, 2017 and February 5, 2018, or before and after the Thomas Fire in Santa Barbara County, which burned from December 4, 2017 through January 12, 2018. The image below shows the fire’s burn perimeter, and the darker, decorrelated areas closely correspond with the USGS's mapped fire perimeter. The Pacific Ocean is the decorrelated, dark area in the lower portion of the image. 

<img src="Thomas.png" width="800" align="center" /> <br> 